# ==============================================
# 2_data_preprocessing.ipynb
# Datenbereinigung, Tokenisierung & Splitting für StyleShift
# ==============================================

# Zelle 1: Importiere benötigte Bibliotheken
import os
import pandas as pd
import re
import json
from sklearn.model_selection import train_test_split
import nltk
from nltk.tokenize import word_tokenize

# Sicherstellen, dass NLTK-Tokenizer verfügbar sind (nur beim ersten Mal)
nltk.download('punkt')

# Definiere Pfade
RAW_DATA_DIR = "../data/raw"
PROCESSED_DATA_DIR = "../data/processed"
os.makedirs(PROCESSED_DATA_DIR, exist_ok=True)

# Zelle 2: Rohdaten laden
# Annahme: Die CSV-Datei "style_texts_raw.csv" enthält zwei Spalten:
# "modern_text" und "shakespeare_text".
raw_file = os.path.join(RAW_DATA_DIR, "style_texts_raw.csv")
df = pd.read_csv(raw_file)
print(f"Anzahl Datensätze: {len(df)}")
display(df.head())

# Zelle 3: Datenbereinigung
def clean_text(text):
    """
    Entfernt HTML-Tags, überflüssige Leerzeichen und trimmt den Text.
    Konvertiert den Text in Kleinbuchstaben.
    """
    if not isinstance(text, str):
        text = str(text)
    text = re.sub(r"<.*?>", "", text)      # HTML-Tags entfernen
    text = re.sub(r"\s+", " ", text)         # Mehrfache Leerzeichen reduzieren
    return text.strip().lower()

# Bereinige beide Stile
df["modern_clean"] = df["modern_text"].apply(clean_text)
df["shakespeare_clean"] = df["shakespeare_text"].apply(clean_text)

# Optional: Entferne sehr kurze Texte (z.B. weniger als 20 Zeichen)
df = df[df["modern_clean"].str.len() > 20]
df = df[df["shakespeare_clean"].str.len() > 20]

print("\nBereinigte Texte:")
display(df[["modern_clean", "shakespeare_clean"]].head())

# Zelle 4: Tokenisierung
def tokenize_text(text):
    """
    Zerlegt den Text in Tokens mittels NLTK.
    """
    return word_tokenize(text)

df["modern_tokens"] = df["modern_clean"].apply(tokenize_text)
df["shakespeare_tokens"] = df["shakespeare_clean"].apply(tokenize_text)

print("\nTokenisierte Beispiele:")
display(df[["modern_tokens", "shakespeare_tokens"]].head())

# Zelle 5: Vokabular erstellen
from collections import Counter

# Kombiniere beide Token-Spalten, um ein gemeinsames Vokabular zu bauen
all_tokens = []
for tokens in df["modern_tokens"]:
    all_tokens.extend(tokens)
for tokens in df["shakespeare_tokens"]:
    all_tokens.extend(tokens)

token_freqs = Counter(all_tokens)
print(f"Anzahl eindeutiger Tokens: {len(token_freqs)}")

# Setze ein Limit für das Vokabular (optional)
VOCAB_SIZE = 10000  # Passe an, je nach Datenmenge
most_common_tokens = token_freqs.most_common(VOCAB_SIZE)

# Definiere Sondertokens
special_tokens = ["<PAD>", "<UNK>", "<BOS>", "<EOS>"]
word2id = {}
idx = 0
for token in special_tokens:
    word2id[token] = idx
    idx += 1

for token, _ in most_common_tokens:
    if token not in word2id:
        word2id[token] = idx
        idx += 1

vocab_size = len(word2id)
print(f"Endgültige Vokabulargröße (inkl. Sondertokens): {vocab_size}")

# Erstelle das inverse Mapping
id2word = {v: k for k, v in word2id.items()}

# Zelle 6: Tokens in IDs umwandeln
def tokens_to_ids(token_list, word2id, unk_id=word2id["<UNK>"]):
    """
    Wandelt eine Liste von Tokens in ihre entsprechenden IDs um.
    """
    return [word2id[t] if t in word2id else unk_id for t in token_list]

df["modern_ids"] = df["modern_tokens"].apply(lambda tokens: tokens_to_ids(tokens, word2id))
df["shakespeare_ids"] = df["shakespeare_tokens"].apply(lambda tokens: tokens_to_ids(tokens, word2id))

# Optional: Füge BOS und EOS hinzu
BOS = word2id["<BOS>"]
EOS = word2id["<EOS>"]
df["modern_ids"] = df["modern_ids"].apply(lambda ids: [BOS] + ids + [EOS])
df["shakespeare_ids"] = df["shakespeare_ids"].apply(lambda ids: [BOS] + ids + [EOS])

print("\nBeispiel für Token-IDs:")
display(df[["modern_ids", "shakespeare_ids"]].head())

# Zelle 7: Train/Val/Test Split
# Bei parallelen Datensätzen kannst du einfach zufällig splitten,
# oder nach bestimmten Kriterien (z.B. nach Kapitel oder Autor).
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

print(f"Train: {len(train_df)}, Val: {len(val_df)}, Test: {len(test_df)}")

# Zelle 8: Speichern der verarbeiteten Daten und des Vokabulars
os.makedirs(PROCESSED_DATA_DIR, exist_ok=True)

# Speichere das Vokabular als JSON
vocab_path = os.path.join(PROCESSED_DATA_DIR, "vocab.json")
with open(vocab_path, "w", encoding="utf-8") as f:
    json.dump(word2id, f, ensure_ascii=False)
print("Vokabular gespeichert:", vocab_path)

# Speichere die DataFrames als CSV-Dateien
train_csv_path = os.path.join(PROCESSED_DATA_DIR, "train.csv")
val_csv_path = os.path.join(PROCESSED_DATA_DIR, "val.csv")
test_csv_path = os.path.join(PROCESSED_DATA_DIR, "test.csv")

train_df.to_csv(train_csv_path, index=False)
val_df.to_csv(val_csv_path, index=False)
test_df.to_csv(test_csv_path, index=False)

print("\nDaten gespeichert:")
print("Train CSV:", train_csv_path)
print("Val CSV:", val_csv_path)
print("Test CSV:", test_csv_path)

# Zelle 9: Ausblick & Fazit
print("""
Die Daten wurden erfolgreich bereinigt, tokenisiert und in Trainings-, Validierungs- und Test-Sets aufgeteilt.
Als nächster Schritt folgt in Notebook 3_training_demo.ipynb das Training eines Modells für den Stiltransfer.
""")
